In [2]:
!pip install trl
!pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 KB 7.1 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.26.1
    Uninstalling accelerate-0.26.1:
      Successfully uninstalled accelerate-0.26.1


In [1]:
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, BitsAndBytesConfig,AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from trl import RewardTrainer, SFTTrainer , AutoModelForSeq2SeqLMWithValueHead,PPOTrainer, PPOConfig
from trl import create_reference_model
from trl.core import LengthSampler
from datasets import Dataset, load_dataset
import numpy as np
import json
import random
import pandas as pd
from transformers import Trainer, TrainingArguments
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model

2024-03-12 15:03:05.065942: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 15:03:05.065988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 15:03:05.067154: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-12 15:03:05.074233: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 15:03:05.824224: W tensorflow/compiler/tf2

In [2]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# Load a tokenizer (change the model name as per your requirements)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
df = pd.read_csv('small_Rlhf.csv')

In [4]:
df.head(2)

,prompt,chosen,rejected
0,SUBREDDIT: r/relationship_advice\nTITLE: [19/M...,TL;DR: Girlfriend thinks she's holding me back...,TL;DR: Girlfriend wants me to experience colle...
1,SUBREDDIT: r/tifu\nTITLE: TIFU by making a pro...,TL;DR: I said a prostitute joke to my friend a...,TL;DR: TIFU by making a prostitute joke


In [5]:
df.shape

(1000, 3)

In [6]:
raw_dataset = Dataset.from_pandas(df)
raw_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})

In [7]:
# Tokenize the dataset and get the lengths
tokenized_lengths = raw_dataset.map(lambda examples: {'lengths': len(tokenizer(examples['chosen'], add_special_tokens=True)["input_ids"])}, remove_columns=raw_dataset.column_names)
# Fetch max length
max_length = max(tokenized_lengths["lengths"])
print("Max token count:", max_length)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Max token count: 116


In [8]:
len(raw_dataset)

1000

In [9]:
raw_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1000
})

In [10]:
# Split the dataset into train, val, and test
train_percent = 0.8
val_percent = 0.1
# test_percent is implicitly 0.1 since train + val + test = 1.0

train_size = int(train_percent * 1000)
val_size = int(val_percent * 1000)

# Remaining samples are for testing
train_dataset = raw_dataset.select(list(range(train_size)))
val_dataset = raw_dataset.select(list(range(train_size, train_size + val_size)))
test_dataset = raw_dataset.select(list(range(train_size + val_size, 1000)))

In [11]:
train_dataset ,val_dataset, test_dataset

(Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 800
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 100
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 100
 }))

In [12]:
def process_features(batch):
    # Tokenize 'chosen' feature
    chosen_tokens = tokenizer(batch['chosen'], padding='max_length', truncation=True, max_length=512, return_tensors='np')
    batch['input_ids_chosen'] = chosen_tokens['input_ids']
    batch['attention_mask_chosen'] = chosen_tokens['attention_mask']

    # Tokenize 'rejected' feature
    rejected_tokens = tokenizer(batch['rejected'], padding='max_length', truncation=True, max_length=512, return_tensors='np')
    batch['input_ids_rejected'] = rejected_tokens['input_ids']
    batch['attention_mask_rejected'] = rejected_tokens['attention_mask']

    return batch

# Apply the function to your datasets
train_dataset = train_dataset.map(process_features, batched=True)
val_dataset = val_dataset.map(process_features, batched=True)

# Remove original 'chosen' and 'rejected' columns
columns_to_remove = ['chosen', 'rejected']
train_dataset = train_dataset.remove_columns(columns_to_remove)
val_dataset = val_dataset.remove_columns(columns_to_remove)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

/tmp/ipykernel_14099/712054621.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/home/azureuser/.local/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [14]:
output_dir = 'rlhf_reward_model/'

training_args = TrainingArguments(
    output_dir=output_dir,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=50,
    logging_steps = 1
)

trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #peft_config=peft_config,
    compute_metrics=compute_metrics,
    max_length=256,
)

/home/azureuser/.local/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:110: FutureWarning: Using `transformers.TrainingArguments` for `args` is deprecated and will be removed in a future version. Please use `RewardConfig` instead.
  warnings.warn(
/home/azureuser/.local/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:115: FutureWarning: The `max_length` argument is deprecated and will be removed in a future version. Please use the `RewardConfig` to set `max_length` instead.
  warnings.warn(
/home/azureuser/.local/lib/python3.10/site-packages/trl/trainer/reward_trainer.py:189: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(


In [15]:
!nvidia-smi

Tue Mar 12 15:03:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000001:00:00.0 Off |                  Off |
| N/A   42C    P0              26W /  70W |   2497MiB / 16384MiB |     22%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/azureuser/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Accuracy
1,0.762900,0.672661,0.560000
2,0.480300,0.759062,0.590000
3,0.216400,1.104303,0.550000
4,0.047100,1.203124,0.560000
5,0.034200,1.325342,0.590000
6,0.001800,1.863919,0.550000
7,0.002000,2.200639,0.520000
8,0.003700,1.718042,0.550000
9,0.029900,1.534842,0.560000
10,0.005000,2.191980,0.600000


Checkpoint destination directory rlhf_reward_model/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/azureuser/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/azureuser/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/azureuser/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/azureuser/.local/lib/python3.10/site-packages/transformers/

TrainOutput(global_step=5000, training_loss=0.0709556962271412, metrics={'train_runtime': 7155.2215, 'train_samples_per_second': 5.59, 'train_steps_per_second': 0.699, 'total_flos': 0.0, 'train_loss': 0.0709556962271412, 'epoch': 50.0})

In [18]:
trainer.state.log_history

[{'loss': 0.684, 'learning_rate': 4.999e-05, 'epoch': 0.01, 'step': 1},
 {'loss': 0.6819,
  'learning_rate': 4.9980000000000006e-05,
  'epoch': 0.02,
  'step': 2},
 {'loss': 0.7034, 'learning_rate': 4.997e-05, 'epoch': 0.03, 'step': 3},
 {'loss': 0.6787, 'learning_rate': 4.996e-05, 'epoch': 0.04, 'step': 4},
 {'loss': 0.7213, 'learning_rate': 4.995e-05, 'epoch': 0.05, 'step': 5},
 {'loss': 0.6909,
  'learning_rate': 4.9940000000000006e-05,
  'epoch': 0.06,
  'step': 6},
 {'loss': 0.6958,
  'learning_rate': 4.9930000000000005e-05,
  'epoch': 0.07,
  'step': 7},
 {'loss': 0.6489, 'learning_rate': 4.992e-05, 'epoch': 0.08, 'step': 8},
 {'loss': 0.7301, 'learning_rate': 4.991e-05, 'epoch': 0.09, 'step': 9},
 {'loss': 0.7351, 'learning_rate': 4.99e-05, 'epoch': 0.1, 'step': 10},
 {'loss': 0.6535,
  'learning_rate': 4.9890000000000005e-05,
  'epoch': 0.11,
  'step': 11},
 {'loss': 0.7034,
  'learning_rate': 4.9880000000000004e-05,
  'epoch': 0.12,
  'step': 12},
 {'loss': 0.6867, 'learning_r

In [20]:
# Save the model and tokenizer
model.save_pretrained("reward_mode_50epochs/")
tokenizer.save_pretrained("reward_model_tokenizer_50epochs/")

('reward_model_tokenizer_50epochs/tokenizer_config.json',
 'reward_model_tokenizer_50epochs/special_tokens_map.json',
 'reward_model_tokenizer_50epochs/vocab.txt',
 'reward_model_tokenizer_50epochs/added_tokens.json',
 'reward_model_tokenizer_50epochs/tokenizer.json')

In [21]:
# Evaluate the model
results = trainer.evaluate()

# Print metrics
print(results)

/home/azureuser/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'eval_loss': 2.442725896835327, 'eval_accuracy': 0.52, 'eval_runtime': 5.5011, 'eval_samples_per_second': 18.178, 'eval_steps_per_second': 2.363, 'epoch': 50.0}


In [22]:
predictions, label_ids, _ = trainer.predict(val_dataset)

/home/azureuser/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [23]:
# Convert logits to labels
predicted_labels = np.argmax(predictions, axis=1)

# Compute accuracy or any other metric
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(label_ids, predicted_labels)
print("Accuracy:", accuracy)

Accuracy: 0.52


In [25]:
test_dataset_for_accuracy = test_dataset.map(process_features, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [26]:
predictions, label_ids, _ = trainer.predict(test_dataset_for_accuracy)
# Convert logits to labels
predicted_labels = np.argmax(predictions, axis=1)

# Compute accuracy or any other metric
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(label_ids, predicted_labels)
print("Accuracy:", accuracy)

/home/azureuser/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Accuracy: 0.51


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [30]:
import torch.nn.functional as F


def score_summaries(model, tokenizer, chosen_summary, rejected_summary):
    # Tokenize the inputs
    chosen_tokens = tokenizer(chosen_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
    rejected_tokens = tokenizer(rejected_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)

    chosen_tokens.to(device)
    rejected_tokens.to(device)


    # Get logits from the model
    with torch.no_grad():
        chosen_logits = model(**chosen_tokens).logits
        rejected_logits = model(**rejected_tokens).logits

    # Apply softmax to get probabilities
    chosen_probs = F.softmax(chosen_logits, dim=-1)
    rejected_probs = F.softmax(rejected_logits, dim=-1)

    # Assuming the positive class (indicating 'chosen' is good) is the second one
    chosen_score = chosen_probs[0][1].item()
    rejected_score = rejected_probs[0][1].item()

    # Extract logits for each summary
    chosen_logit = chosen_logits[0][1].item()
    rejected_logit = rejected_logits[0][1].item()

    return chosen_score, rejected_score, chosen_logit, rejected_logit

In [31]:
# Example usage
chosen_summary = "TL;DR: Water meter in another condo is not in our condo. What can we do legally to restore water to my condo complex?"
rejected_summary = "TL;DR: I don't know"

chosen_score, rejected_score, chosen_logit, rejected_logit = score_summaries(model, tokenizer, chosen_summary, rejected_summary)

print(f"Chosen Score: {chosen_score:.4f}")
print(f"Rejected Score: {rejected_score:.4f}")

print(f"Chosen Logit: {chosen_logit:.4f}")
print(f"Rejected Logit: {rejected_logit:.4f}")

Chosen Score: 0.5441
Rejected Score: 0.4987
Chosen Logit: 6.1556
Rejected Logit: -6.4933


In [35]:
test_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 100
})

In [40]:
for i in test_dataset['chosen']:
    print(i)

TL;DR: My boyfriend cheated on me by having an intimate Skype convo with another girl over webcam after 4+ years together. I can't forgive him and don't feel the same way about him anymore.
TL;DR: Girlfriend is having second thoughts about our relationship, and I'm worried that I'm setting myself up for pain. Am I setting myself up for pain?
TL;DR: found out my boyfriend cheated on me over webcam with a girl he got off a webcam site.. I am having a lot of trouble forgiving him and I don't know what to do.
TL;DR: The only good reason I can think of for humans to be in space is because we think it's important to say we are there.
TL;DR: Ex of 7 months broke up with me, she apologized and asked if we could be friends, i agreed, things were great till last Thursday she began to ignore me, dont know why. Help pls
TL;DR: I'm looking for ideas to propose to my boyfriend, and I don't know what to do to make it not awkward.
TL;DR: boyfriend stopped having sex, says nothing is wrong, I'm not sur

In [46]:
chosen_summary = []
rejected_summary = []
for i,j in zip(test_dataset['chosen'], test_dataset['rejected']):
    chosen_summary.append(i)
    rejected_summary.append(j)

for chosen, rejected in zip(chosen_summary, rejected_summary):
    chosen_score, rejected_score, chosen_logit, rejected_logit = score_summaries(model, tokenizer, chosen, rejected)

    print(f"Chosen Score: {chosen_score:.4f}")
    print(f"Rejected Score: {rejected_score:.4f}")

    print(f"Chosen Logit: {chosen_logit:.4f}")
    print(f"Rejected Logit: {rejected_logit:.4f}")
    print("\n\n\n")

Chosen Score: 0.5618
Rejected Score: 0.5136
Chosen Logit: 6.2911
Rejected Logit: -5.9902




Chosen Score: 0.4982
Rejected Score: 0.4950
Chosen Logit: -5.8289
Rejected Logit: -1.4790




Chosen Score: 0.4849
Rejected Score: 0.4907
Chosen Logit: 1.3715
Rejected Logit: -6.4591




Chosen Score: 0.4962
Rejected Score: 0.4974
Chosen Logit: -6.4856
Rejected Logit: -5.4297




Chosen Score: 0.5636
Rejected Score: 0.5458
Chosen Logit: 1.3878
Rejected Logit: -1.2086




Chosen Score: 0.5298
Rejected Score: 0.5078
Chosen Logit: 1.3085
Rejected Logit: -6.5759




Chosen Score: 0.4918
Rejected Score: 0.5505
Chosen Logit: -3.2979
Rejected Logit: -2.2035




Chosen Score: 0.5141
Rejected Score: 0.4688
Chosen Logit: 6.0325
Rejected Logit: 1.6186




Chosen Score: 0.5646
Rejected Score: 0.5182
Chosen Logit: 6.5027
Rejected Logit: -4.5801




Chosen Score: 0.5215
Rejected Score: 0.5338
Chosen Logit: 1.8814
Rejected Logit: 1.1466




Chosen Score: 0.5360
Rejected Score: 0.5515
Chosen Logit: 3.3674
Reje

In [45]:
len(chosen_summary)

100